In [ ]:
using ProgressMeter, PyCall, PyPlot, Images, HDF5,NaNStatistics, Statistics, DSP, Lasso, JLD2
using _Data, _Math

In [ ]:
@pyimport numpy
@pyimport scipy.stats as stats
@pyimport matplotlib.colors as mpl_colors
@pyimport matplotlib.cm as cm 

In [ ]:
rc_params = PyDict(pyimport("matplotlib")["rcParams"]);
rc_params["font.sans-serif"] = ["Arial"];
rc_params["font.size"] = 7;
rc_params["lines.linewidth"] = 1;
rc_params["lines.markersize"] = 4;
rc_params["xtick.major.size"] = 2;
rc_params["ytick.major.size"] = 2;
rc_params["xtick.major.pad"] = 2;
rc_params["ytick.major.pad"] = 2;
# rc_params["axes.labelpad"] = 2;
rc_params["axes.spines.right"] = false
rc_params["axes.spines.top"] = false

cim(img::Matrix{UInt32}) = CairoImageSurface(img, Cairo.FORMAT_RGB24; flipxy = false) 
cim(img::Matrix{ARGB32}) = cim(reinterpret(UInt32, img))
cim(img::Matrix{RGB24}) = cim(reinterpret(UInt32, img))
cim(img::Matrix{UInt8}) = cim(Gray.(reinterpret(N0f8, img)))
cim(img::Array{UInt8,3}) = cim(RGB24.(reinterpret(N0f8, img[:,:,1]), reinterpret(N0f8, img[:,:,2]), reinterpret(N0f8, img[:,:,3])));downsample(img,n=4) = +([img[i:n:n*div(size(img,1)-1,n), j:n:n*div(size(img,2)-1,n)] for i = 1:n, j = 1:n]...)/(n*n);
downsample(img,n=4) = +([img[i:n:n*div(size(img,1)-1,n), j:n:n*div(size(img,2)-1,n)] for i = 1:n, j = 1:n]...)/(n*n);

In [ ]:
include("../../../functions/func_map.jl")
include("../../../functions/func_stat.jl")
include("../../../functions/func_data.jl")
include("../../../functions/func_plot.jl")

In [ ]:
data_info_dict = load("/home/chuyu/fish_place_cell_paper/figures/chuyu/figure_data_info.jld2")

data_info_all = []
data_info_tog = []
job_names = []
for key in keys(data_info_dict)
    append!(data_info_all, data_info_dict[key])
    append!(data_info_tog, [data_info_dict[key]])
    append!(job_names, [String(key)])
end

In [ ]:
analyzer = "chuyu"

In [ ]:
using Printf
function plot_mean_density(bins, y_tog; plot_color="k", shade_color="gray", line_color="k", plot_median = true)
    
    n_bins = length(bins)
    x = (bins[1:end-1] + bins[2:end])/2
    y = mean(y_tog)
    ymax = y + std(y_tog)
    ymin = y - std(y_tog);
    
    cumsum_y_mean = cumsum(mean(y_tog)*(maximum(bins) - minimum(bins))/(n_bins-1))

    x_median = numpy.round(numpy.interp(0.5, cumsum_y_mean, x), 2)
    x_14 = numpy.round(numpy.interp(0.25, cumsum_y_mean, x), 2)
    x_34 = numpy.round(numpy.interp(0.75, cumsum_y_mean, x), 2)
    
    
    plot(x, y, color= plot_color)
    fill_between(x, ymax, ymin, color=shade_color, alpha=0.3)
    
    # println(minimum(ymin) , " ", maximum(ymax))
    if plot_median
        axvline(x_median, color=line_color, label= @sprintf("%.2f",x_median), linestyle="dashed")
        # axvline(x_median, color=line_color, label= "$x_median ($x_14 - $x_34)")
    end
    
    
    return ymin, ymax
end

In [ ]:
job_names_ordered = [
 "corner_cue_rotation_whole"
 "landmark_removal"
 "boundary_morphing"
    
    
 "job_retangular_half_circle_rotation"

    
 "hedgehog_rotation_bottom_out"
"hedgehog_landmark_removal"
 "hedgehog_circle"
 "corner_cue_circle"
    
]


In [ ]:
job_names_figure = [
 "Chamber rotation"
 "Landmark removal"
 "Wall morphing"
    
 "Wall rotation"
    
 "No change"
 "Landmark removal"
 "Wall morphing"
 "Wall morphing + \n Landmark removal"
]


# match angle to best

In [ ]:
fig_path = "/home/chuyu/Notebooks/project_place_cell/figures/output/sfigure10/summary"
mkpath(fig_path)

In [ ]:
job_names_ordered

In [ ]:
job_names

In [ ]:
job_names_figure

In [ ]:
fig_path

In [ ]:
fig = figure(figsize=(1.8,1))
tick_locs = []
tick_labels = []
for (i_job, which_job) in enumerate([3, 7, 8])
data_info_all = data_info_dict[job_names_ordered[which_job]]
job_name = job_names_figure[which_job]
println(job_name)
n_bins = 31

anglematch_median_tog = []
best_median_tog = []
siginficance_all = []
plot_y_all = []     

for which_data in 1:length(data_info_all)
    data_info = data_info_all[which_data]
    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]
    experiment_filename_2 = data_info[3]
    server_2 = data_info[4]
        
    experimenter = data_info[end]

    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    
    
     ds_save_cy_2 = Dataset(experiment_filename_2, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_2)")
       
        
    if job_name != "boundary_morphing"
        info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_matchangle.h5")
    else
        info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_matchboundary.h5") 
    end
    file = h5open(info_filename, "r")
    tel_place_cell_index = HDF5.readmmap(file["tel_place_cell_index"])
    corr_trials_all = HDF5.readmmap(file["corr_trials_all"])
    corr_pv_all = HDF5.readmmap(file["corr_pv_all"])
    corr_pv_all_normalized = HDF5.readmmap(file["corr_pv_all_normalized"])
    peak_shift_mm = HDF5.readmmap(file["peak_shift_mm"])
    close(file)
        

    append!(anglematch_median_tog, nanmedian(corr_trials_all[tel_place_cell_index]))

        
        
    if job_name != "boundary_morphing"
        info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results.h5")
    else
        info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results.h5") 
    end
    file = h5open(info_filename, "r")
    tel_place_cell_index = HDF5.readmmap(file["tel_place_cell_index"])
    corr_trials_all = HDF5.readmmap(file["corr_trials_all"])
    corr_pv_all = HDF5.readmmap(file["corr_pv_all"])
    corr_pv_all_normalized = HDF5.readmmap(file["corr_pv_all_normalized"])
    peak_shift_mm = HDF5.readmmap(file["peak_shift_mm"])
    close(file)
    


    
    append!(best_median_tog, nanmedian(corr_trials_all[tel_place_cell_index]))
    



    control_file = h5open(joinpath(data_path(ds_save_cy_2), "boundary_morph_random.h5"))
    p_value = 1 - read(control_file,"significance")
    close(control_file)
        
    siginficance = "n.s."
    if p_value < 0.01
        siginficance = "*"
    end
    if p_value < 1e-3
        siginficance = "**"
    end
    if p_value < 1e-5
        siginficance = "***"
    end        
        
    println(siginficance)
    formatted_string = @sprintf("%.3E", p_value)
    formatted_string = replace(formatted_string, "E" => ".E")
    println(formatted_string)

        
    # println(siginficance)
    append!(siginficance_all, [siginficance])
    append!(plot_y_all, best_median_tog[which_data] - 0.04)     

end


for i in 1:length(best_median_tog)
    plot([i_job*2-1,i_job*2], [anglematch_median_tog[i], best_median_tog[i]], color="gray", alpha=0.5)
    scatter([i_job*2-1], anglematch_median_tog[i], color="blue", alpha=0.7, label = "Wall referred registration")
    scatter([i_job*2], best_median_tog[i], color="red", alpha=0.7, label = "Best wall referred registration")

end
    
    
    println(mean(best_median_tog .- anglematch_median_tog))
    println(std(best_median_tog .- anglematch_median_tog))
    

yticks([0,0.3,0.6])


end
xticks([])
ylabel("PF correlation")



In [ ]:
fig = figure(figsize=(1.8,1))
tick_locs = []
tick_labels = []
for (i_job, which_job) in enumerate([3, 7, 8])
data_info_all = data_info_dict[job_names_ordered[which_job]]
job_name = job_names_figure[which_job]

n_bins = 31

anglematch_median_tog = []
best_median_tog = []
    

for which_data in 1:length(data_info_all)
    data_info = data_info_all[which_data]
    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]
     experimenter = data_info[3]

    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    
    
    if job_name != "boundary_morphing"
        info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_matchangle.h5")
    else
        info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_matchboundary.h5") 
    end
    file = h5open(info_filename, "r")
    tel_place_cell_index = HDF5.readmmap(file["tel_place_cell_index"])
    corr_trials_all = HDF5.readmmap(file["corr_trials_all"])
    pv_map_normalized = HDF5.readmmap(file["pv_map_dFF"])
    corr_pv_all_normalized = pv_map_normalized[isfinite.(pv_map_normalized)]
    peak_shift_mm = HDF5.readmmap(file["peak_shift_mm"])
    close(file)
        

    append!(anglematch_median_tog, nanmedian(corr_pv_all_normalized))

        
        
    if job_name != "boundary_morphing"
        info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results.h5")
    else
        info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results.h5") 
    end
    file = h5open(info_filename, "r")
    tel_place_cell_index = HDF5.readmmap(file["tel_place_cell_index"])
    corr_trials_all = HDF5.readmmap(file["corr_trials_all"])
    pv_map_normalized = HDF5.readmmap(file["pv_map_dFF"])
    corr_pv_all_normalized = pv_map_normalized[isfinite.(pv_map_normalized)]
    peak_shift_mm = HDF5.readmmap(file["peak_shift_mm"])
    close(file)
    


    
    append!(best_median_tog, nanmedian(corr_pv_all_normalized))
    

    
    
end
println(anglematch_median_tog)
println(best_median_tog)


for i in 1:length(best_median_tog)
    plot([i_job*2-1,i_job*2], [anglematch_median_tog[i], best_median_tog[i]], color="gray", alpha=0.5)
        
    scatter([i_job*2-1], anglematch_median_tog[i], color="blue", alpha=0.7, label = "Wall referred registration")
    scatter([i_job*2], best_median_tog[i], color="red", alpha=0.7, label = "Best wall referred registration")

        
end
    println(mean(best_median_tog .- anglematch_median_tog))
    println(std(best_median_tog .- anglematch_median_tog))

    
yticks([0,0.3,0.6])

end

xticks(tick_locs, tick_labels, rotation=90)
ylabel("PV correlation")


In [ ]:
fig = figure(figsize=(1.8,1))
tick_locs = []
tick_labels = []
for (i_job, which_job) in enumerate([3, 7, 8])
data_info_all = data_info_dict[job_names_ordered[which_job]]
job_name = job_names_figure[which_job]

n_bins = 31

anglematch_median_tog = []
best_median_tog = []
    

for which_data in 1:length(data_info_all)
    data_info = data_info_all[which_data]
    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]
     experimenter = data_info[3]

    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    
    
    if job_name != "boundary_morphing"
        info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_matchangle.h5")
    else
        info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results_matchboundary.h5") 
    end
    file = h5open(info_filename, "r")
    tel_place_cell_index = HDF5.readmmap(file["tel_place_cell_index"])
    corr_trials_all = HDF5.readmmap(file["corr_trials_all"])
    corr_pv_all = HDF5.readmmap(file["corr_pv_all"])
    corr_pv_all_normalized = HDF5.readmmap(file["corr_pv_all_normalized"])
    peak_shift_mm = HDF5.readmmap(file["peak_shift_mm"])
    close(file)
        

    append!(anglematch_median_tog, nanmedian(peak_shift_mm))

        
        
    if job_name != "boundary_morphing"
        info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results.h5")
    else
        info_filename = joinpath(data_path(ds_save_cy_1), "compare_map_results.h5") 
    end
    file = h5open(info_filename, "r")
    tel_place_cell_index = HDF5.readmmap(file["tel_place_cell_index"])
    corr_trials_all = HDF5.readmmap(file["corr_trials_all"])
    corr_pv_all = HDF5.readmmap(file["corr_pv_all"])
    corr_pv_all_normalized = HDF5.readmmap(file["corr_pv_all_normalized"])
    peak_shift_mm = HDF5.readmmap(file["peak_shift_mm"])
    close(file)
    


    
    append!(best_median_tog, nanmedian(peak_shift_mm))
    

    
    
end


for i in 1:length(best_median_tog)
    plot([i_job*2-1,i_job*2], [anglematch_median_tog[i], best_median_tog[i]], color="gray", alpha=0.5)
        
    scatter([i_job*2-1], anglematch_median_tog[i], color="blue", alpha=0.7, label = "Wall referred registration")
    scatter([i_job*2], best_median_tog[i], color="red", alpha=0.7, label = "Best wall referred registration")
    if i == 1 && i_job == 1
        legend(frameon=false, loc=(1,0.6))
    end
        
end

    println(mean(best_median_tog .- anglematch_median_tog))
    println(std(best_median_tog .- anglematch_median_tog))


end

xticks(tick_locs, tick_labels, rotation=90)
ylabel("PF shift (mm)")

yticks([0,10,20])


In [ ]:
fig_path